In [ ]:
import pandas as pd
from emle_bespoke.utils import create_dimer_topology
from openff.interchange import Interchange as _Interchange
from openff.toolkit import ForceField as _ForceField
import numpy as np
import openmm as mm
import openmm.unit as unit
from openmm import app
import torch

In [ ]:
df = pd.read_csv('/home/joaomorado/repos/emle-bespoke/utils/DESS66x8.csv')

In [ ]:
from collections import defaultdict
import numpy as np

# Pre-allocate lists for results
results = defaultdict(list)

# Initialize force field and interchange object
force_field = _ForceField("openff-2.0.0.offxml")

# Get unique system IDs
unique_ids = df["system_id"].unique()

# Loop over unique IDs
for system_id in unique_ids:
    # Filter DataFrame for the current system ID
    df_filtered = df[df["system_id"] == system_id]
    smiles0, smiles1 = df_filtered.iloc[0][["smiles0", "smiles1"]]
    natoms0, natoms1 = df_filtered.iloc[0][["natoms0", "natoms1"]]

    # Determine solute and solvent based on oxygen atom "O"
    if smiles0 == "O": 
        solute, solvent = smiles1, smiles0
        solvent_idx = list(range(natoms0))
        solute_idx = list(range(natoms0, natoms0 + natoms1))
    elif smiles1 == "O":
        solute, solvent = smiles0, smiles1
        solute_idx = list(range(natoms0))
        solvent_idx = list(range(natoms0, natoms0 + natoms1))
    else:
        solute, solvent = smiles0, smiles1
        solute_idx = list(range(natoms0))
        solvent_idx = list(range(natoms0, natoms0 + natoms1))

    
    # Debugging print statement (optional)
    print(f"Processing system: {system_id}, Solute: {solute}, Solvent: {solvent}")
    # Create topology and extract molecules
    topology = create_dimer_topology(solute, solvent)
    mols = list(topology.molecules)

    # Precompute atomic numbers and masks
    solute_atomic_numbers = [atom.atomic_number for atom in mols[0].atoms]
    solvent_atomic_numbers = [atom.atomic_number for atom in mols[1].atoms]
    atomic_numbers = np.array(solute_atomic_numbers + solvent_atomic_numbers, dtype=int)

    # Create solute and solvent masks
    solute_mask = np.arange(len(atomic_numbers)) < len(solute_atomic_numbers)
    solvent_mask = ~solute_mask

    # Create interchange and extract charges
    interchange = _Interchange.from_smirnoff(force_field=force_field, topology=topology)
    system = interchange.to_openmm()
    nonbonded_force = next(
        f for f in system.getForces() if isinstance(f, mm.NonbondedForce)
    )
    charges = np.array([
        nonbonded_force.getParticleParameters(i)[0].value_in_unit(
            nonbonded_force.getParticleParameters(i)[0].unit
        )
        for i in range(system.getNumParticles())
    ])
    charges_mm = charges[solvent_mask]

    # Process each row in the filtered DataFrame
    for _, row in df_filtered.iterrows():
        # Parse XYZ coordinates
        xyz = np.fromstring(row["xyz"], sep=" ").reshape(-1, 3)
        energy = row["cbs_CCSD(T)_all"] * 4.184

        # Append computed results to respective lists
        results["e_int_list"].append(energy)
        results["xyz_mm_list"].append(xyz[solvent_idx])
        results["xyz_qm_list"].append(xyz[solute_idx])
        results["atomic_numbers_list"].append(np.asarray(solute_atomic_numbers))
        results["charges_mm_list"].append(charges_mm)
        results["solute_mask_list"].append(solute_mask)
        results["solvent_mask_list"].append(solvent_mask)
        results["topology_list"].append(topology)

# Save to pickle file
import pickle
with open("dess66x8.pkl", "wb") as f:
    pickle.dump(results, f)

# Convert results dictionary to individual lists if necessary
e_int_list = results["e_int_list"]
xyz_mm_list = results["xyz_mm_list"]
xyz_qm_list = results["xyz_qm_list"]
atomic_numbers_list = results["atomic_numbers_list"]
charges_mm_list = results["charges_mm_list"]
solute_mask_list = results["solute_mask_list"]
solvent_mask_list = results["solvent_mask_list"]


In [ ]:
xyz_list = [np.concatenate([xyz_qm, xyz_mm], axis=0) for xyz_qm, xyz_mm in zip(xyz_qm_list, xyz_mm_list)]



In [ ]:
# Write to xyz
import os
import numpy as np
from emle_bespoke._constants import ATOMIC_NUMBERS_TO_SYMBOLS


with open("file.xyz", "w") as f:
    for xyz in xyz_list:
        f.write(f"{len(xyz)}\n")
        f.write("\n")
        for atom, coord in zip(atomic_numbers, xyz):
            symbol = ATOMIC_NUMBERS_TO_SYMBOLS[atom]
            f.write(f"{symbol} {coord[0]:.6f} {coord[1]:.6f} {coord[2]:.6f}\n")



In [ ]:
# Create the Lennard-Jones potential
from emle_bespoke.lj_fitting import LennardJonesPotential as _LennardJonesPotential

lj_potential = _LennardJonesPotential(
    topology_off=results["topology_list"],
    forcefield=force_field,
    parameters_to_fit={
        "all": {"sigma": True, "epsilon": True},
        #"n-tip3p-O": {"sigma": True, "epsilon": True},
    },
    device=torch.device("cuda"),
)

In [ ]:
from emle.train._utils import pad_to_max

In [ ]:
solvent_mask = pad_to_max(solvent_mask_list).to(device=lj_potential._device)
solute_mask = pad_to_max(solute_mask_list).to(device=lj_potential._device)

# Get xyz(batchn,nqm+nmm,3) and atomic_numbers(batchn,nqm+nmm)
xyz = [np.concatenate([xyz_qm, xyz_mm], axis=0) for xyz_qm, xyz_mm in zip(xyz_qm_list, xyz_mm_list)]
xyz = pad_to_max(xyz) * 0.1
xyz = torch.tensor(xyz, device=lj_potential._device)

In [ ]:
e_lj_initial = lj_potential(xyz, solute_mask, solvent_mask).detach().cpu().numpy()
print(e_lj_initial)

In [ ]:

from emle.train._utils import pad_to_max
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64
atomic_numbers = pad_to_max(atomic_numbers_list).to(device=device, dtype=torch.int64)
xyz_qm = pad_to_max(xyz_qm_list).to(device=device, dtype=dtype) * 0.1
xyz_mm = pad_to_max(xyz_mm_list).to(device=device, dtype=dtype) * 0.1
charges_mm = pad_to_max(charges_mm_list).to(device=device, dtype=dtype)
e_int = torch.tensor(e_int_list).to(device=device, dtype=dtype)


from emle.models import EMLE
from emle_bespoke._constants import HARTREE_TO_KJ_MOL
emle = EMLE(
    device=device,
    dtype=dtype,
)
e_static, e_ind = emle(atomic_numbers, charges_mm, xyz_qm * 10.0, xyz_mm * 10.0)
e_int_predicted = e_static + e_ind
e_int_predicted = e_int_predicted * HARTREE_TO_KJ_MOL
e_int_predicted = e_int_predicted.detach().cpu().numpy()

In [ ]:
print(atomic_numbers)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the Seaborn style for a clean look
sns.set(style="whitegrid", palette="muted")

# Create a figure and axis
plt.figure(figsize=(10, 6))
print(e_lj_initial)
print(e_int_predicted)

# Plot the energy values as a line plot
plt.plot(e_lj_initial + e_int_predicted, label="EMLE", color="blue")
plt.plot(e_int_list, label="QM", color="red") 

# Add labels and title
plt.xlabel('Time or Iterations', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.title('Energy vs Iterations', fontsize=14)

# Add grid lines for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# Add a legend to the plot
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Print shapes
print("atomic_numbers shape:", atomic_numbers.shape)
print("xyz_qm shape:", xyz_qm.shape)
print("xyz_mm shape:", xyz_mm.shape)
print("charges_mm shape:", charges_mm.shape)
print("e_int shape:", e_int.shape)
print("solute_mask shape:", solute_mask.shape)
print("solvent_mask shape:", solvent_mask.shape)


In [ ]:

torch.autograd.detect_anomaly(True)
from emle_bespoke.bespoke import BespokeModelTrainer as _BespokeModelTrainer

# Fit the Lennard-Jones potential
emle_bespoke = _BespokeModelTrainer(device=device, dtype=dtype)

emle_bespoke.fit_lj(
    lj_potential=lj_potential,
    xyz_qm=xyz_qm,
    xyz_mm=xyz_mm,
    xyz=xyz,
    atomic_numbers=atomic_numbers,
    charges_mm=charges_mm,
    e_int_target=e_int,
    solute_mask=solute_mask,
    solvent_mask=solvent_mask,
    lr=0.001,
    epochs=1000,
)

In [ ]:
lj_potential.print_lj_parameters(lj_potential._lj_params)

In [ ]:

e_lj_final = lj_potential(xyz, solute_mask, solvent_mask).detach().cpu().numpy()




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the Seaborn style for a clean look
sns.set(style="whitegrid", palette="muted")

# Create a figure and axis
plt.figure(figsize=(10, 6))

# Plot the energy values as a line plot
plt.plot(e_int_predicted + e_lj_final, label='Predicted', color='blue', linewidth=2)
plt.plot(e_int_predicted + e_lj_initial, label='Initial', color='green', linewidth=2)
plt.plot(e_int_list, label='Target', color='red', linewidth=2)


# Add labels and title
plt.xlabel('Time or Iterations', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.title('Energy vs Iterations', fontsize=14)

# Add grid lines for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# Add a legend to the plot
plt.legend()
plt.ylim(-40, 40)
# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Assuming _atom_type_to_index is a dictionary
atom_type_to_index = lj_potential._atom_type_to_index

# Reverse the dictionary
index_to_atom_type = {v: k for k, v in atom_type_to_index.items()}

# Now you can access atom types by index
print(index_to_atom_type)

atom_types = [index_to_atom_type[i+1] for i in range(len(index_to_atom_type))]


In [ ]:

sigma_init = lj_potential._sigma_init.detach().cpu().numpy().squeeze()
sigma_final = lj_potential._sigma.detach().cpu().numpy().squeeze()
epsilon_init = lj_potential._epsilon_init.detach().cpu().numpy().squeeze() 
epsilon_final = lj_potential._epsilon.detach().cpu().numpy().squeeze()


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Example data (replace with your data)
atom_indices = np.arange(len(sigma_init))
delta_sigma = sigma_final - sigma_init  # Change in sigma
delta_epsilon = epsilon_final - epsilon_init  # Change in epsilon

# Create a DataFrame for Seaborn
import pandas as pd
data = pd.DataFrame({
    'Atom Index': np.tile(atom_indices, 2),
    'Delta': np.concatenate([delta_sigma, delta_epsilon]),
    'Parameter': ['Delta Sigma'] * len(atom_indices) + ['Delta Epsilon'] * len(atom_indices)
})

# Plot
plt.figure(figsize=(12, 8))
sns.barplot(data=data, x='Atom Index', y='Delta', hue='Parameter', palette="viridis", edgecolor='black')
plt.xlabel("Atom Index", fontsize=14)
plt.ylabel("Change (Δ)", fontsize=14)
plt.title("Change in Sigma and Epsilon Across Atom Indices", fontsize=16, weight='bold')
plt.xticks(ticks=np.arange(len(atom_types))+1, labels=atom_types, fontsize=12, rotation=45)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(title="Parameter", fontsize=12)
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Example data (replace with your own data)
# e_int_list = [...]  # QM Energies
# e_int_predicted = [...]  # Predicted energies
# e_lj_initial = [...]  # Initial LJ energies
# e_lj_final = [...]  # Final LJ energies

# Set Seaborn style for better aesthetics
sns.set(style="whitegrid", context="talk")

# Create the plot
plt.figure(figsize=(8, 7))
plt.scatter(e_int_list, e_int_predicted + e_lj_initial, color='blue', alpha=0.7, label='Initial LJ')
plt.scatter(e_int_list, e_int_predicted + e_lj_final, color='red', alpha=0.7, label='Final LJ')

# Add identity line for reference
min_energy = min(e_int_list)
max_energy = max(e_int_list)
plt.plot([-1000,1000], [-1000, 1000], color='gray', linestyle='--', label='y=x')

# Labels and title
plt.xlabel("QM Energy (kJ/mol)", fontsize=14)
plt.ylabel("EMLE Energy (kJ/mol)", fontsize=14)
plt.title("QM vs EMLE Energies", fontsize=16, weight='bold')

# Legend and grid
plt.legend(fontsize=12, loc='best')
plt.grid(alpha=0.3)

# Adjust ticks for readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.xlim(-100,100)
plt.ylim(-100,100)

# Show plot
plt.tight_layout()
plt.show()
